# Notes for reimplemtation
## Goals
Fit charges for multiple molecules and multiple conformation
Fit polarization for multiple molecule and multiple conformations
Fit charges and polarizabilities for multiple conf/ molecules
Fit BCCs for multiple molecules
Fit BCCs and polarizaibilities for multiple molecule and conformations
## Things to consider
Charges are a per atom property
BCCs are a bond property and is limited by the number of different bonds in the offxml file
Polarizations parameters are limited by the number of atom types I allow in the forcefield file

For BCC POL and BCCPOL the logic idea is to build a matrix from size (len(bcc)+len(pol)) and buid this matrix 
for every molecule independently. The drawback is that a lot of the entries are zero but it is super is to 
parallize with dask.

For charges the the number of matrix elemnts is not obvious. Every charge gets its own row. Therefore the same approach is impractical. I can still follow the route of building one matrix and then combine the matrices with 
lagrange multiplicators. 

I want to build the matrix from the bottom up. So make sure for polarized molecules to build the matrix for 1 conformation including all ESP (base and pol) first and then go on. Speedup of a factor of 7 possible

include timing

include logging

Include units via package Pint

## Open Questions
Do I want to include more than 1 molecule per ESP --> probably not now. 


## Classes

### Traingset
1 class per instance. Defines what type of optimization we are performing. 

vector qpol

calc qpol
update qpol

write results to offxml

- parmeters:
    - tzpe of optimization (q/d/qd/bcc/bccpol/pol)
    
- attributes:
    - the main matrix which is optimized (at least for bccpol and pol) 

### Molecule
Constist of multiple conformations.
 
 init(mol2):
    self.offmol = openforcefield.Molecule(mol2)
    self._T = self._create_T()
    self._qll = self._create_qll()
    self._Tpol = self._create_Tpol()
    self._S = self._create_S()
    self._conformers = list() # Important that we don't take confs from mol2
    
    
 
 openff.Molecule
 qll 
 T
 Tpol
 ScalingMatrix S
 conformers
 
properties:
- T
- qll
- Tpol

functions:
- build_A
- build_B
- get_charges_including_bccs(bccvector)
- add_conformer_from_mol2(self, mol2)
    Check atom ordering
    get coordinates from mol2
    self._conformers.append(coordinates)


### Conformation

def __init__(self, mol2_file):
   self._atom_positions = (get atom positions here)

attributes: 
    baseESP
    polESPs
    atompositions (natoms*3 numpy array)
    
functions:
    distances = get_distances()
    combine_grids()
    del_cached_data()*
    add_grid
    

# Creating the data structure example
dataset = MolDataSet()
for mol_file in top_folder:
  new_mol = Molecule(mol_file)
  for conf_file in folder_2:
    new_conf = Conformer(conf_file)
    for esp_file, grid_file in folder_3:
      e_field = (somehow figure out efield here)
      new_conf.add_grid(esp_file, grid_file, e_field)
    new_mol.add_conformer(new_conf)
  dataset.add_molecule(new_mol)
    


### Atom
    ext_efield

### Bond

### (possibly) GridCollection
info:
  grid_positions


### ESPGRID
def __init__(self, esp_file, grid_file):
  self._esp_values = load_grid_file(esp_file)
  self._grid_positions = load_positions(grid_file)
  
    Information:
    esp_values (Pint-wrapped numpy array)
    grid_positions (Pint-wrapped n*3 numpy array)
    @property Efield vector
    
    Functions:
    None
    
##### BCCUnpolESP (ESPGRID)
     
##### BCCPolESP (ESPGRID)
def __init__(self, esp_file, grid_file, ext_field):
  super().__init__(esp_file, grid_file)
  self._ext_field = ext_field
    ext_field
    
      
## Functions

### equil_atoms
Returns the atomic number of equivalent atoms (intramolecular)




### Input files required
List of atom names

### Open Questions:
 - How to i link the molecules and the esp file in the input. 
     - Should i think about a predefinied order structure 
 - Should I split the trainingset class into 2 subclasses one for charges and one for bccs
     
